# Importing libraries

In [11]:
import numpy as np
from fitizens_libraries.load_and_process_training_data import load_training_data
import os
import pandas as pd
import statistics as stat
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten,Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import F1Score
from sklearn.model_selection import ParameterGrid

# Creating Functions

In [12]:
from enum import Enum
class ExerciseType(Enum):
    BURPEE = "BURPEE"
    DUMBBELL_SNATCH = "DUMBBELL_SNATCH"
    JUMPING_JACK = "JUMPING_JACK"
    SQUAT_JUMP = "SQUAT_JUMP"
    PULL_UP = "PULL_UP"
    PUSH_UP = "PUSH_UP"
    SIT_UP = "SIT_UP"
    SQUAT = "SQUAT"

class ExerciseFolder(Enum):
    BURPEE = "BURPEE"
    DUMBBELL_SNATCH = "DUMBBELL"
    JUMPING_JACK = "JACK"
    SQUAT_JUMP = "JUMPS"
    PULL_UP = "PULLUP"
    PUSH_UP = "PUSHUP"
    SIT_UP = "SITUP"
    SQUAT = "SQUATS"

In [13]:
class ExerciseInfo:
    def __init__(self, exercise: ExerciseType, folder_path: ExerciseFolder):
        self.exercise = exercise.value
        self.folder_path = f"data/{folder_path.value}"

In [14]:
def requires_peak_minima(exercise: str):
    if exercise == ExerciseType.PULL_UP.value or exercise == ExerciseType.SIT_UP.value or exercise == ExerciseType.JUMPING_JACK.value:
        return False
    else:
        return True

In [15]:
def load_data(exercise: str, folder_path: str):
    os.makedirs(folder_path, exist_ok=True)
    file_names = [f"{folder_path}/{name}" for name in os.listdir(folder_path)]
    signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
    
    data, wk = load_training_data(filelist=file_names,
                             signals= signals,
                              target_exercise=exercise, other_exercises=[], is_peak_minima=requires_peak_minima(exercise))
    print(f"'{exercise}' data loaded")
    return data

In [16]:
def resampling_data(df_input, desired_rows, columns):
    df = df_input[columns]
    if df.shape[0] == desired_rows:
        return df    

    additional_index = pd.date_range(start=df.index.min(),  
                                     end=df.index.max(),
                                     periods=desired_rows)
    resampled_df = df.reindex(additional_index).interpolate()
    
    return resampled_df

In [17]:
def get_training_test_data(exercise_tag: str, folder_path: str, columns_selected: list):
    data = load_data(exercise_tag, folder_path)
    
    exercise = [element["series"] for element in list(filter(lambda info: info["target"] == exercise_tag, data))]
    no_exercise = [element["series"] for element in list(filter(lambda info: info["target"] == "NO_EXERCISE", data))]
    
    exercise_len = len(exercise)
    no_exercise_len = len(no_exercise)
    total_len = exercise_len + no_exercise_len
    exercise_weight = total_len / ( 2 * exercise_len )
    no_exercise_weight = total_len / ( 2* no_exercise_len )
    
    metric = int(stat.median([info.shape[0] for info in exercise]))
    resampled_exercise = [resampling_data(info, metric, columns_selected) for info in exercise]
    resampled_no_exercise = [resampling_data(info, metric, columns_selected) for info in no_exercise]
    
    resampled_exercise_labeled = [1] * len(resampled_exercise)
    resampled_no_exercise_labeled = [0] * len(resampled_no_exercise)
    
    input_data = resampled_exercise + resampled_no_exercise
    input_data = np.stack([df.to_numpy() for df in input_data])
    output_data = np.array(resampled_exercise_labeled + resampled_no_exercise_labeled)
    
    return {"X": input_data, "Y": output_data, "exercise_weight": exercise_weight, "no_exercise_weight": no_exercise_weight, "metric": metric}

In [18]:
def create_model(params, input_shape):
    model = Sequential()
    model.add(Conv1D(filters=params['filters'], kernel_size=params['kernel_size'], 
                     activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Dense(params['dense_units'], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(Dense(2, activation='softmax'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=[F1Score()])
    return model

In [19]:
def hyperparameter_tuning(X_train, y_train, X_test, y_test, input_shape, class_weights_dict):
    param_grid = {
        'filters': [32, 64, 128],
        'kernel_size': [3, 5],
        'dense_units': [32, 64, 128],
        'learning_rate': [0.001, 0.0001]
    }

    best_f1 = 0
    best_model = None
    best_params = None

    for params in ParameterGrid(param_grid):
        model = create_model(params, input_shape)
        model.fit(X_train, y_train, epochs=100, batch_size=32, class_weight=class_weights_dict, validation_split=0.2, verbose=0)
        _, f1 = model.evaluate(X_test, y_test, verbose=0)
        f1_exercise = f1[1]
        if f1_exercise > best_f1:
            best_f1 = f1_exercise
            best_model = model
            best_params = params
        print(f"F1 Iteration: {f1_exercise}")

    return best_model, best_f1, best_params

In [20]:
def predict_deep_learning(exercise_tag: str, folder_path: str, columns_selected: list):
    print(f"Predict {exercise_tag}")
    data = get_training_test_data(exercise_tag, folder_path, columns_selected)
    
    X_train, X_test, y_train, y_test = train_test_split(data["X"], data["Y"], test_size=0.2, random_state=123)
    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)
    class_weights_dict = {0: data["no_exercise_weight"], 1: data["exercise_weight"]}
    input_shape = (data["metric"], len(columns_selected))

    return hyperparameter_tuning(X_train, y_train, X_test, y_test, input_shape, class_weights_dict)

# Creating models

In [21]:
features = ["accX", "accY", "accZ", "linAccX", "linAccY", "linAccZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ"]

models = [ExerciseInfo(ExerciseType.BURPEE, ExerciseFolder.BURPEE),
          ExerciseInfo(ExerciseType.DUMBBELL_SNATCH, ExerciseFolder.DUMBBELL_SNATCH),
          ExerciseInfo(ExerciseType.JUMPING_JACK, ExerciseFolder.JUMPING_JACK),
          ExerciseInfo(ExerciseType.SQUAT_JUMP, ExerciseFolder.SQUAT_JUMP),
          ExerciseInfo(ExerciseType.PULL_UP, ExerciseFolder.PULL_UP),
          ExerciseInfo(ExerciseType.PUSH_UP, ExerciseFolder.PUSH_UP),
          ExerciseInfo(ExerciseType.SIT_UP, ExerciseFolder.SIT_UP),
          ExerciseInfo(ExerciseType.SQUAT, ExerciseFolder.SQUAT)]

outcomes = []
for model in models:
    outcome = {}
    model_name = model.exercise
    
    best_model_generated, best_f1_generated, best_params_generated = predict_deep_learning(model_name, model.folder_path, features)
    
    outcome["model_name"] = model_name
    outcome["f1"] = best_f1_generated
    outcome["dense_units"] = best_params_generated["dense_units"]
    outcome["filters"] = best_params_generated["filters"]
    outcome["kernel_size"] = best_params_generated["kernel_size"]
    outcome["learning_rate"] = best_params_generated["learning_rate"]
    
    outcomes.append(outcome)
    
    best_model_generated.save(f"models/{model_name}.keras")

outcomes_df = pd.DataFrame(outcomes)

Predict BURPEE
'BURPEE' data loaded
F1 Iteration: 0.9473684430122375
F1 Iteration: 0.8944099545478821
F1 Iteration: 0.8735632300376892
F1 Iteration: 0.9032257795333862
F1 Iteration: 0.8588234782218933
F1 Iteration: 0.9044585824012756
F1 Iteration: 0.8622754812240601
F1 Iteration: 0.9426751136779785
F1 Iteration: 0.9125000238418579
F1 Iteration: 0.923076868057251
F1 Iteration: 0.1355932056903839
F1 Iteration: 0.8607594966888428
F1 Iteration: 0.9036144018173218
F1 Iteration: 0.9127516746520996
F1 Iteration: 0.9240505695343018
F1 Iteration: 0.9419354796409607
F1 Iteration: 0.1355932056903839
F1 Iteration: 0.8789809346199036
F1 Iteration: 0.1355932056903839
F1 Iteration: 0.9150327444076538
F1 Iteration: 0.8260869979858398
F1 Iteration: 0.9411764144897461
F1 Iteration: 0.1355932056903839
F1 Iteration: 0.9102563261985779
F1 Iteration: 0.9440994262695312
F1 Iteration: 0.8831168413162231
F1 Iteration: 0.8957054615020752
F1 Iteration: 0.9342105388641357
F1 Iteration: 0.9342105388641357
F1 Itera

In [22]:
outcomes_df

,model_name,f1,dense_units,filters,kernel_size,learning_rate
0,BURPEE,0.947368,32,32,3,0.0010
1,DUMBBELL_SNATCH,0.990741,32,128,5,0.0001
2,JUMPING_JACK,0.945205,64,32,3,0.0010
3,SQUAT_JUMP,0.957983,64,32,5,0.0010
4,PULL_UP,0.593750,128,64,5,0.0010
5,PUSH_UP,0.785714,64,64,3,0.0001
6,SIT_UP,0.729412,128,64,5,0.0001
7,SQUAT,0.978979,128,64,3,0.0001
